# Convolutional RBM (cRBM)

This notebook takes care of implementing the basic functionality for cRBMs.
Or maybe it's just for the preliminaries, that is some simple stuff before it actually comes to the Boltzmann Machine.


## Part 1: Reading the data and converting it to various forms of matrices

In [12]:
import theano.tensor as T
import theano.tensor.nnet.conv as conv
import numpy as np
import Bio.SeqIO as sio
import Bio.motifs.matrix as mat
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from Bio import motifs

import random
import time

### Classes to read biological files (such as FASTA or JASPAR)

In [4]:
"""
This class reads sequences from fasta files.
To use it, create an instance of that object and use
the function readSequencesFromFile.
"""
class FASTAReader:
    
    def __init__(self, _path):
        self.path = _path
        
    def readSequencesFromFile (self, filename):
        dhsSequences = []
        for dhs in sio.parse(open(filename), 'fasta', IUPAC.unambiguous_dna):
            dhsSequences.append(dhs.seq)
        return dhsSequences
    
    
class JASPARReader:
    
    def __init__ (self):
        pass
    
    def readSequencesFromFile (self, filename):
        matrices = []
        for mat in motifs.parse(open(filename), 'jaspar'):
            matrices.append(mat.pwm)
        return matrices

In [5]:
matReader = JASPARReader()
pwms = matReader.readSequencesFromFile('data/jaspar_matrices.txt')

In [6]:
# apply the two classes to calculate a forward pass in our algorithm
seqReader = FASTAReader('.')
allSeqs = seqReader.readSequencesFromFile('data/wgEncodeAwgDnaseUwAg10803UniPk.fa')

In [7]:
test_set = [allSeqs[random.randrange(0,len(allSeqs))] for i in range(1000)]
print len(test_set)

1000


In [15]:
def getIntToLetter (letter):
    if letter == 'A' or letter == 'a':
        return 0
    elif letter == 'C' or letter == 'c':
        return 1
    elif letter == 'G' or letter == 'g':
        return 2
    elif letter == 'T' or letter == 't':
        return 3
    else:
        print "ERROR. LETTER " + letter + " DOES NOT EXIST!"
        return -1

def getMatrixFromSeq (seq):
    m = len(seq.alphabet.letters)
    n = len(seq)
    result = np.zeros((1, m, n))
    revSeq = seq.reverse_complement()
    for i in range(len(seq)):
        result[0,getIntToLetter(seq[i]),i] = 1
    return result


start = time.time()
dataMat = np.array([getMatrixFromSeq(t) for t in test_set])
print "Conversion of test set in (in ms): " + str((time.time()-start)*1000)

Conversion of test set in (in ms): 152.47297287


## Part 2a: Borrowing Ian Goodfellow's implementation of the probabilistic max pooling layer
This implementation is now part of the pylearn2 library which is licensed under the 3-claused BSD license.
Source code is available here: https://github.com/lisa-lab/pylearn2/blob/master/pylearn2/expr/probabilistic_max_pooling.py

In [16]:

from theano.gof.op import get_debug_values

def max_pool(z, pool_shape, top_down=None, theano_rng=None):
    """
    Probabilistic max-pooling
    Parameters
    ----------
    z : theano 4-tensor
        a theano 4-tensor representing input from below
    pool_shape : tuple
        tuple of ints. the shape of regions to be pooled
    top_down : theano 4-tensor, optional
        a theano 4-tensor representing input from above
        if None, assumes top-down input is 0
    theano_rng : MRG_RandomStreams, optional
        Used for random numbers for sampling
    Returns
    -------
    p : theano 4-tensor
        the expected value of the pooling layer p
    h : theano 4-tensor
        the expected value of the detector layer h
    p_samples : theano 4-tensor, only returned if theano_rng is not None
        samples of the pooling layer
    h_samples : theano 4-tensor, only returned if theano_rng is not None
        samples of the detector layer
    Notes
    ------
    all 4-tensors are formatted with axes ('b', 'c', 0, 1).
    This is for maximum speed when using theano's conv2d
    to generate z and top_down, or when using it to infer conditionals of
    other layers using the return values.
    Detailed description:
    Suppose you have a variable h that lives in a Conv2DSpace h_space and
    you want to pool it down to a variable p that lives in a smaller
    Conv2DSpace p.
    This function does that, using non-overlapping pools.
    Specifically, consider one channel of h. h must have a height that is a
    multiple of pool_shape[0] and a width that is a multiple of pool_shape[1].
    A channel of h can thus be broken down into non-overlapping rectangles
    of shape pool_shape.
    Now consider one rectangular pooled region within one channel of h.
    I now use 'h' to refer just to this rectangle, and 'p' to refer to
    just the one pooling unit associated with that rectangle.
    We assume that the space that h and p live in is constrained such
    that h and p are both binary and p = max(h). To reduce the state-space
    in order to make probabilistic computations cheaper we also
    constrain sum(h) <= 1.
    Suppose h contains k different units. Suppose that the only term
    in the model's energy function involving h is -(z*h).sum()
    (elemwise multiplication) and the only term in
    the model's energy function involving p is -(top_down*p).sum().
    Then P(h[i] = 1) = softmax( [ z[1], z[2], ..., z[k], -top_down] )[i]
    and P(p = 1) = 1-softmax( [z[1], z[2], ..., z[k], -top_down])[k]
    This variation of the function assumes that z, top_down, and all
    return values use Conv2D axes ('b', 'c', 0, 1).
    This variation of the function implements the softmax using a
    theano graph of exp, maximum, sub, and div operations.
    Performance notes:
    It might be possible to make a faster implementation with different
    theano ops. rather than using set_subtensor, it might be possible
    to use the stuff in theano.sandbox.neighbours. Probably not possible,
    or at least nasty, because that code isn't written with multiple
    channels in mind, and I don't think just a reshape can fix it.
    Some work on this in galatea.cond.neighbs.py
    At some point images2neighbs' gradient was broken so check that
    it has been fixed before sinking too much time into this.
    Stabilizing the softmax is also another source of slowness.
    Here it is stabilized with several calls to maximum and sub.
    It might also be possible to stabilize it with
    T.maximum(-top_down,T.signal.downsample.max_pool(z)).
    Don't know if that would be faster or slower.
    Elsewhere in this file I implemented the softmax with a reshape
    and call to Softmax / SoftmaxWithBias.
    This is slower, even though Softmax is faster on the GPU than the
    equivalent max/sub/exp/div graph. Maybe the reshape is too expensive.
    Benchmarks show that most of the time is spent in GpuIncSubtensor
    when running on gpu. So it is mostly that which needs a faster
    implementation. One other way to implement this would be with
    a linear.Conv2D.lmul_T, where the convolution stride is equal to
    the pool width, and the thing to multiply with is the hparts stacked
    along the channel axis. Unfortunately, conv2D doesn't work right
    with stride > 2 and is pretty slow for stride 2. Conv3D is used to
    mitigate some of this, but only has CPU code.
    """

    z_name = z.name
    if z_name is None:
        z_name = 'anon_z'

    batch_size, ch, zr, zc = z.shape

    r, c = pool_shape

    zpart = []

    mx = None

    if top_down is None:
        t = 0.
    else:
        t = - top_down
        t.name = 'neg_top_down'

    for i in xrange(r):
        zpart.append([])
        for j in xrange(c):
            cur_part = z[:, :, i:zr:r, j:zc:c]
            if z_name is not None:
                cur_part.name = z_name + '[%d,%d]' % (i, j)
            zpart[i].append(cur_part)
            if mx is None:
                mx = T.maximum(t, cur_part)
                if cur_part.name is not None:
                    mx.name = 'max(-top_down,' + cur_part.name + ')'
            else:
                max_name = None
                if cur_part.name is not None:
                    mx_name = 'max(' + cur_part.name + ',' + mx.name + ')'
                mx = T.maximum(mx, cur_part)
                mx.name = mx_name
    mx.name = 'local_max(' + z_name + ')'

    pt = []

    for i in xrange(r):
        pt.append([])
        for j in xrange(c):
            z_ij = zpart[i][j]
            safe = z_ij - mx
            safe.name = 'safe_z(%s)' % z_ij.name
            cur_pt = T.exp(safe)
            cur_pt.name = 'pt(%s)' % z_ij.name
            pt[-1].append(cur_pt)

    off_pt = T.exp(t - mx)
    off_pt.name = 'p_tilde_off(%s)' % z_name
    denom = off_pt

    for i in xrange(r):
        for j in xrange(c):
            denom = denom + pt[i][j]
    denom.name = 'denom(%s)' % z_name

    off_prob = off_pt / denom
    p = 1. - off_prob
    p.name = 'p(%s)' % z_name

    hpart = []
    for i in xrange(r):
        hpart.append([pt_ij / denom for pt_ij in pt[i]])

    h = T.alloc(0., batch_size, ch, zr, zc)

    for i in xrange(r):
        for j in xrange(c):
            h.name = 'h_interm'
            h = T.set_subtensor(h[:, :, i:zr:r, j:zc:c], hpart[i][j])

    h.name = 'h(%s)' % z_name

    if theano_rng is None:
        return p, h
    
    ### --------------------- DONE IF NO SAMPLES ARE GENERATED ---------------------------###
    else:
        events = []
        for i in xrange(r):
            for j in xrange(c):
                events.append(hpart[i][j])
        events.append(off_prob)

        events = [event.dimshuffle(0, 1, 2, 3, 'x') for event in events]

        events = tuple(events)

        stacked_events = T.concatenate(events, axis=4)

        rows = zr // pool_shape[0]
        cols = zc // pool_shape[1]
        outcomes = pool_shape[0] * pool_shape[1] + 1
        assert stacked_events.ndim == 5
        for se, bs, r, c, chv in get_debug_values(stacked_events, batch_size,
                                                  rows, cols, ch):
            assert se.shape[0] == bs
            assert se.shape[1] == r
            assert se.shape[2] == c
            assert se.shape[3] == chv
            assert se.shape[4] == outcomes
        reshaped_events = stacked_events.reshape((
            batch_size * rows * cols * ch, outcomes))

        multinomial = theano_rng.multinomial(pvals=reshaped_events,
                                             dtype=p.dtype)

        reshaped_multinomial = multinomial.reshape((batch_size, ch, rows,
                                                    cols, outcomes))

        h_sample = T.alloc(0., batch_size, ch, zr, zc)

        idx = 0
        for i in xrange(r):
            for j in xrange(c):
                h_sample = T.set_subtensor(h_sample[:, :, i:zr:r, j:zc:c],
                                           reshaped_multinomial[:, :, :, :,
                                           idx])
                idx += 1

        p_sample = 1 - reshaped_multinomial[:, :, :, :, -1]

        return p, h, p_sample, h_sample



In [71]:
## PART 3: Optimizing theano to do it all on the GPU


from theano.tensor.shared_randomstreams import RandomStreams


class CRBM:

    def __init__ (self, _motifLength, _numMotifs, _learningRate=0.1, _poolingFactor=1, _alphabet=IUPAC.unambiguous_dna):
        # parameters for the motifs
        self.motifLength = _motifLength
        self.numMotifs = _numMotifs
        self.alphabet = _alphabet
        self.initializeMotifs()
        
        # cRBM parameters
        self.bias = theano.shared(value=np.random.rand(self.numMotifs), name='bias', borrow=True)
        self.c = theano.shared(value=np.array([random.random()]), name='c', borrow=True)
        self.poolingFactor = _poolingFactor
        self.learningRate = _learningRate
        
        # infrastructural parameters
        self.rng = np.random.RandomState()
        self.theano_rng = RandomStreams(self.rng.randint(2**30))
    
    
    def initializeMotifs (self):
        # create random kernel
        x = np.random.rand(2 * self.numMotifs, 1, 4, self.motifLength).astype(np.float32)
        
        # create reverse complement
        for i in range(self.numMotifs):
            x[self.numMotifs+i] = np.flipud(np.fliplr(x[self.numMotifs+i]))
            
        self.motifs = theano.shared(value=x, name='W', borrow=True)
        
        
    def setCustomKernels (self, customKernels):
        if len(customKernels.shape) != 4 or customKernels.shape[1] != 1:
            print "New motifs must be a 4D matrix with dims: (K x 1 x numOfLetters(4) x numOfKMers)"
            return
        
        self.numMotifs = customKernels.shape[0]
        self.motifLength = customKernels.shape[3]
        self.bias = theano.shared(value=np.random.rand(self.numMotifs), name='bias', borrow=True)
        self.motifs = theano.shared(value=customKernels.astype(np.float32))
        print "New motifs set. # Motifs: " + str(self.numMotifs) + " K-mer-Length: " + str(self.motifLength)

        
### ------------------------------THE TOUGH STUFF-------------------------------- ###
### ----------------------------------------------------------------------------- ###

    def forwardBatch (self, data):
        out = conv.conv2d(data, self.motifs)[:,:,::-1,::-1] # flip, because conv reverts H
        bMod = self.bias.dimshuffle('x', 0, 'x', 'x') # add dims to the bias until it works
        pooled = max_pool(out.dimshuffle(0,2,1,3), pool_shape=(2, self.poolingFactor), theano_rng=self.theano_rng)
        return [pooled[1], pooled[3]] #only return pooled layer and probs


    def backwardBatch (self, H_sample):
        # theano convolution call
        #K_star = self.motifs.dimshuffle(1, 0, 2, 3)[:,:,::-1,::-1]
        C = conv.conv2d(H_sample, self.motifs[:,:,:,::-1], border_mode='full')
        out = T.sum(C, axis=1, keepdims=True) # sum over all K
        #out = out + self.c
        
        # add fourth dimension (the strands) that was lost during forward pass
        #res = self.softmax(out).dimshuffle(0,'x',1,2)
        return out


    def gradient (self, hiddenProbs, data):
        mean = T.mean(hiddenProbs, axis=0, keepdims=True) # sum over all samples to get avg (but keep dim)
        mean = T.tile(mean, [2,1,1,1])
        H_reshaped = mean.dimshuffle(1, 0, 2, 3)
        out = conv.conv2d(data, H_reshaped)
        return T.mean(out, axis=0)

    
    def batchTraining (self, data, epochs, batchSize, numOfCDs):
        
        # calculate the data gradient for weights (motifs) and bias
        D_batch = T.tensor4('data')
        [H_data, S_data] = self.forwardBatch(D_batch)
        G_data = self.gradient(H_data, D_batch)
        bias_data = T.mean(T.sum(H_data, axis=3), axis=0)
        
        # calculate the model gradient (only CD-1 for now)
        V_model = self.backwardBatch(S_data)
        S_v = self.sampleVisibleLayer(V_model)
        [H_model, S_model] = self.forwardBatch(S_v)
        G_model = self.gradient(H_model, D_batch)
        bias_model = T.mean(T.sum(H_model, axis=3), axis=0)
        
        self.motifs = self.motifs + self.learningRate * (G_data - G_model)
        self.bias = self.bias + self.learningRate * (bias_data - bias_model)
        
        training = theano.function([D_batch], [self.motifs,self.bias], allow_input_downcast=True)
        
        print "START"
        
        # after we got the theano function for everything, let's apply it!
        itPerEpoch = data.shape[0] / batchSize
        for epoch in range(epochs):
            for batch in range(itPerEpoch):
                print batch*batchSize, (batch+1)*batchSize
                training = training(data[batch*batchSize:(batch+1)*batchSize])
            print "Epoch done: " + str(epoch)
        
        return training
        
        
        
 
    def sampleVisibleLayer (self, V):
        S = self.theano_rng.multinomial(n=1,pvals=V.dimshuffle(0,1,3,2)).dimshuffle(0,1,3,2)
        S = S.astype('float32')
        return S
    
    def softmax (self, x):
        return T.exp(x) / T.exp(x).sum(axis=1, keepdims=True)

In [78]:
theano.config.exception_verbosity='high'
theano.config.optimizer='None'
theano.config.compute_test_value='ignore'

x = CRBM(4,2,0.1, 3)
kernel1 = np.tile(np.array([[1,0,0],[0,1,0],[0,0,1],[0,0,0]]), [1,1,1])
kernel1_ = np.tile(np.flipud(np.fliplr(kernel1[0])),[1,1,1])
print kernel1_
#kernel2 = np.tile(np.array([[0,0,0],[0,0,0],[1,1,1],[0,0,0]]), [1,1,1])
#kernel1 = np.tile(np.eye(4), [2,1,1])
#kernel2 = np.tile(np.array([[0,0,0,0],[0,0,0,0],[1,1,1,1],[0,0,0,0]]), [2,1,1])
kernel = np.array([kernel1, kernel1_])
print kernel.shape

randSeq1 = getMatrixFromSeq(Seq("ACGTGGGG", IUPAC.unambiguous_dna))
randSeq2 = getMatrixFromSeq(Seq("ACGTACGT", IUPAC.unambiguous_dna))
data = np.array([randSeq1, randSeq2])
print "Data shape: " + str(data.shape)
x.setCustomKernels(kernel)
print x.motifs.shape

# my custom theano function testing ground
D = T.tensor4('data')
[P,H,P_S,S] = max_pool(conv.conv2d(D, x.motifs).dimshuffle(0,2,1,3), pool_shape=(2,3), theano_rng=x.theano_rng)
H = H.dimshuffle(0,2,1,3)
S = S.dimshuffle(0,2,1,3)
V = x.backwardBatch(H)
test = theano.function([D], [H,V], allow_input_downcast=True)
result = test(data)
print result[1]
print result[0]
print result[0].shape
print result[1].shape



#print "Data mat shape: " + str(dataMat.shape)
#res = x.batchTraining(data, 1, 1, 10)
#print "Result from training: "
#print res[0]
#print res[1]

[[[0 0 0]
  [1 0 0]
  [0 1 0]
  [0 0 1]]]
(2, 1, 4, 3)
Data shape: (2, 1, 4, 8)
New motifs set. # Motifs: 2 K-mer-Length: 3
Shape.0


ValueError: the filter stack size (1) and image stack size (2) differ
Apply node that caused the error: ConvOp{('imshp', (None, None, None)),('kshp', (None, None)),('nkern', None),('bsize', None),('dx', 1),('dy', 1),('out_mode', 'full'),('unroll_batch', None),('unroll_kern', None),('unroll_patch', True),('imshp_logical', (None, None, None)),('kshp_logical', (None, None)),('kshp_logical_top_aligned', True)}(DimShuffle{0,2,1,3}.0, Subtensor{::, ::, ::, ::int64}.0)
Inputs types: [TensorType(float32, 4D), TensorType(float32, 4D)]
Inputs shapes: [(2, 2, 1, 6), (2, 1, 4, 3)]
Inputs strides: [(48, 24, 24, 4), (48, 48, 12, -4)]
Inputs values: ['not shown', 'not shown']

Backtrace when the node is created:
  File "<ipython-input-71-4f2c73d69fbc>", line 63, in backwardBatch
    C = conv.conv2d(H_sample, self.motifs[:,:,:,::-1], border_mode='full')

Debugprint of the apply node: 
ConvOp{('imshp', (None, None, None)),('kshp', (None, None)),('nkern', None),('bsize', None),('dx', 1),('dy', 1),('out_